In [ ]:
import numpy as np
import pandas as pd
from urllib.request import urlopen 
import requests
import json
import pprint
import os

#get all data
#Physical interaction data
data = pd.read_csv(r"C:\Users\elvka\.Desktopreal\university\Thesis\Data\Global_PPIN\Interaction_Data\BIOGRID-MV-Physical-4.4.233.tab3\Physical_Interactions.csv")

#Protein Existence data Kuster
url = "https://www.proteomicsdb.org/proteomicsdb/logic/api_v2/api.xsodata/Protein?$top=100000&$select=UNIQUE_IDENTIFIER&$filter=(PROTEIN_EXISTENCE%20eq%201%20and%20TAXCODE%20eq%209606)&$format=json"
response = urlopen(url)
data_json = json.loads(response.read())

#clean kuster data
data_json = data_json['d']['results']


#turn into DataFrame
df = pd.DataFrame(data)
dfprot = pd.DataFrame(data_json)


#number of (human) interactions
print('Number of interactions: ', len(df.index))
df = df[df['Organism ID Interactor A'] == 9606]
df = df[df['Organism ID Interactor B'] == 9606]
print('All interactions in humans: ', len(df.index))


#number of unique interactors
protA = set(df['BioGRID ID Interactor A'])
protB = set(df['BioGRID ID Interactor B'])

print('Interactors A: ', len(protA))
print('Interactors B: ', len(protB))


#check if null values in interactors
number_null = df['BioGRID ID Interactor A'].isnull().values.any()
print('Gibt es null values in Interactors A? ', number_null)

number_null_2 = df['BioGRID ID Interactor B'].isnull().values.any()
print('Gibt es null values in Interactors B? ', number_null_2)


#total proteins involved in interactions
all_prot = protA | protB
print('Anzahl der unique proteins, die an Interactions beteiligt sind: ', len(all_prot))


#gibt es null values in swissprot?
number_null_swiss = df['SWISS-PROT Accessions Interactor A'].isnull().values.any()
print('Null values in swiss-prot? ', number_null_swiss)

number_null_swiss_2 = df['SWISS-PROT Accessions Interactor B'].isnull().values.any()
print('Null values in swiss-prot? ', number_null_swiss_2)


#run BioGRID interactions against Kuster
df = df[df['SWISS-PROT Accessions Interactor A'].isin(dfprot['UNIQUE_IDENTIFIER'])]
df = df[df['SWISS-PROT Accessions Interactor B'].isin(dfprot['UNIQUE_IDENTIFIER'])]
print('Number of interactions after double-check: ', len(df.index))

#number of unique proteins after PaxDB
protA = set(df['BioGRID ID Interactor A'])
protB = set(df['BioGRID ID Interactor B'])
all_prot = protA | protB
print('Anzahl der unique proteins, die an Interactions beteiligt sind (nach Kuster): ', len(all_prot))

#save new file to laptop
df.to_csv('Global_cleaned_PPIN_Kuster.csv')

notebook_path = os.getcwd()
print(notebook_path)